In [ ]:
import gc
import pandas as pd
import numpy as np
import scipy.stats as stats
import math
import plotly.graph_objects as go
import random
import os
import copy
import matplotlib.pyplot as plt

from time import time
from datetime import datetime

import torch
import torch.nn as nn
import torch.nn.functional as F

#!pip install properscoring
import properscoring as ps

#!pip install wandb
import wandb

In [ ]:
print(pd.__version__)
print(np.__version__)
print(torch.__version__)
print(ps.__version__)
print(wandb.__version__)

# Data loading & data preprocessing

In [ ]:
# Load monthly MM data.
# Transform the data into a lists of arrays. Each inner array represents a timeseries.
# Remove all the NaN values from the datasets.

# M4
trainset = pd.read_csv('https://raw.githubusercontent.com/Mcompetitions/M4-methods/master/Dataset/Train/Monthly-train.csv')
testset = pd.read_csv('https://raw.githubusercontent.com/Mcompetitions/M4-methods/master/Dataset/Test/Monthly-test.csv')
trainset.set_index('V1', inplace = True)
testset.set_index('V1', inplace = True)
# Add the testset columns behind the trainset columns
testset_merge = trainset.merge(testset, on = 'V1', how = 'inner')
# Get the data in numpy representation
trainset_np = trainset.values
testset_np = testset_merge.values
# Select all non NaN values from the trainset
trainset_clean = [x[x == x] for x in trainset_np]
# Train/validation/test --------------------------------- NBeats paper validation strategy
testset_m4m = [x[x == x] for x in testset_np]
valset_m4m = trainset_clean.copy()
trainset_m4m = [x[:-18] for x in trainset_clean]

del(trainset, testset, testset_merge, trainset_np, testset_np, trainset_clean)

# Optimization criteria

In [ ]:
# mean_forecasts = batch x fl
# var_forecasts = batch x fl
# targets = batch x fl
# actuals_train = batch x bl

# For training purposes
def GaussianNLL(mean_forecasts, var_forecasts, targets):
    NLL_items = torch.log(2*math.pi*var_forecasts)/2 + (1/(2*var_forecasts)) * (targets - mean_forecasts)**2
    NLL_items_clamped = torch.clamp(NLL_items, -1e2, 1e2)#-1e3, 1e3)
    NLL = torch.mean(NLL_items_clamped)
    return NLL

def GaussianKL(means_p, vars_p, means_q, vars_q):
    var_ratios = vars_p / vars_q
    t1s = ((means_p - means_q)**2 / vars_q)
    KL_items = 0.5 * (var_ratios + t1s - 1 - var_ratios.log())
    KL_items_clamped = torch.clamp(KL_items, 0, 1e2)#1e3)
    KL = torch.mean(KL_items_clamped)
    return KL

def SMAPE_train(mean_forecasts, targets):
    loss = 200 * torch.mean(torch.abs(targets - mean_forecasts) / (torch.abs(mean_forecasts).detach() + torch.abs(targets) + 1e-6))
    return loss

def RMSE(mean_forecasts, targets):
    loss_items = torch.sqrt(torch.mean((targets - mean_forecasts)**2, dim = -1) + 1e-6)
    loss = torch.mean(loss_items)
    return loss

def GaussianWD(means_p, vars_p, means_q, vars_q):
    loss_items = torch.sqrt(torch.mean((means_p - means_q)**2 + vars_p + vars_q - 2*torch.sqrt(vars_p)*torch.sqrt(vars_q), dim = -1) + 1e-6)
    loss = torch.mean(loss_items)
    return loss

# For evaluation purposes
def SMAPE(mean_forecasts, targets):
    loss = 200 * np.mean(np.abs(targets - mean_forecasts) / (np.abs(mean_forecasts) + np.abs(targets)))
    return loss

def MAPE(mean_forecasts, targets):
    loss = 100 * np.mean(np.abs(targets - mean_forecasts) / targets)
    return loss

def RMSSE(mean_forecasts, targets, actuals_train):

    mask = np.abs(actuals_train)>1e-6
    scaling_denom = np.sum(mask, axis = -1) - 1
    scaling_denom = np.where(scaling_denom > 0, scaling_denom, np.ones_like(scaling_denom))

    scaling_data = (actuals_train[:, :, 1:] - actuals_train[:, :, :-1])**2
    scaling_factor = np.sum(scaling_data, axis = -1) / scaling_denom
    scaling_factor = np.where(scaling_factor > 0, scaling_factor, np.ones_like(scaling_factor))

    scaling_factor_reshaped = np.expand_dims(scaling_factor, -1)
    scaling_factor_reshaped = np.tile(scaling_factor_reshaped, targets.shape[-1])

    loss_items = np.sqrt(np.mean((targets - mean_forecasts)**2 / scaling_factor_reshaped, axis = -1))
    loss = np.mean(loss_items)
    return loss

def sCRPS(mean_forecasts, var_forecasts, targets, actuals_train):

    mask = np.abs(actuals_train)>1e-6
    scaling_denom = np.sum(mask, axis = -1) - 1
    scaling_denom = np.where(scaling_denom > 0, scaling_denom, np.ones_like(scaling_denom))

    scaling_data = np.abs(actuals_train[:, :, 1:] - actuals_train[:, :, :-1])
    scaling_factor = np.sum(scaling_data, axis = -1) / scaling_denom
    scaling_factor = np.where(scaling_factor > 0, scaling_factor, np.ones_like(scaling_factor))

    scaling_factor_reshaped = np.expand_dims(scaling_factor, -1)
    scaling_factor_reshaped = np.tile(scaling_factor_reshaped, targets.shape[-1])

    sd_forecasts = np.sqrt(var_forecasts)
    loss_items = np.mean(ps.crps_gaussian(targets, mean_forecasts, sd_forecasts) / scaling_factor_reshaped, axis = -1)
    loss = np.mean(loss_items)
    return loss

def GaussianKL_eval(means_p, vars_p, means_q, vars_q):
    var_ratios = vars_p / vars_q
    t1s = ((means_p - means_q)**2 / vars_q)
    KL_items = 0.5 * (var_ratios + t1s - 1 - np.log(var_ratios))
    KL = np.mean(KL_items)
    return KL

# NBeats models

Generic gaussian building block:

In [ ]:
class GenericNBeatsGaussianBlock(nn.Module):

    def __init__(self,
                 device,
                 backcast_length,
                 forecast_length,
                 hidden_layer_units,
                 thetas_dims,
                 share_thetas,
                 dropout = False,
                 dropout_p = 0.0,
                 neg_slope = 0.00):

        super().__init__()
        self.device = device
        self.backcast_length = backcast_length
        self.forecast_length = forecast_length
        if isinstance(hidden_layer_units, int):
            self.hidden_layer_units = [hidden_layer_units for FC_layer in range(4)]
        else:
            #assert(len(hidden_layer_units) == 4)
            self.hidden_layer_units = hidden_layer_units
        self.thetas_dims = thetas_dims
        self.share_thetas = share_thetas
        self.dropout = dropout
        self.dropout_p = dropout_p
        self.neg_slope = neg_slope

        # shared layers in block
        self.fc1 = nn.Linear(self.backcast_length,
                             self.hidden_layer_units[0])#, bias = False)
        self.fc2 = nn.Linear(self.hidden_layer_units[0], self.hidden_layer_units[1])#, bias = False)
        self.fc3 = nn.Linear(self.hidden_layer_units[1], self.hidden_layer_units[2])#, bias = False)
        self.fc4 = nn.Linear(self.hidden_layer_units[2], self.hidden_layer_units[3])#, bias = False)

        # do not use F.dropout as you want dropout to only affect training (not evaluation mode)
        # nn.Dropout handles this automatically
        if self.dropout:
            self.dropoutlayer = nn.Dropout(p = self.dropout_p)

        # task specific (backcast & forecast) layers in block
        # do not include bias - see section 3.1 - Ruben does include bias for generic blocks
        if self.share_thetas:
            self.theta_bc = self.theta_mean_fc = self.theta_var_fc = nn.Linear(self.hidden_layer_units[3], self.thetas_dims)#, bias = False)
        else:
            self.theta_bc = nn.Linear(self.hidden_layer_units[3], self.thetas_dims)#, bias = False)
            self.theta_mean_fc = nn.Linear(self.hidden_layer_units[3], self.thetas_dims)#, bias = False)
            self.theta_var_fc = nn.Linear(self.hidden_layer_units[3], self.thetas_dims)#, bias = False)

        # block output layers
        self.backcast_out = nn.Linear(self.thetas_dims, self.backcast_length)#, bias = False) # include bias - see section 3.3
        self.mean_forecast_out = nn.Linear(self.thetas_dims, self.forecast_length)#, bias = False) # include bias - see section 3.3
        self.var_forecast_out = nn.Linear(self.thetas_dims, self.forecast_length)#, bias = False) # include bias - see section 3.3


    def forward(self, x):

        if self.dropout:

            h1 = F.leaky_relu(self.fc1(x.to(self.device)), negative_slope = self.neg_slope)
            h1 = self.dropoutlayer(h1)
            h2 = F.leaky_relu(self.fc2(h1), negative_slope = self.neg_slope)
            h2 = self.dropoutlayer(h2)
            h3 = F.leaky_relu(self.fc3(h2), negative_slope = self.neg_slope)
            h3 = self.dropoutlayer(h3)
            h4 = F.leaky_relu(self.fc4(h3), negative_slope = self.neg_slope)

            theta_b = F.leaky_relu(self.theta_bc(h4), negative_slope = self.neg_slope)
            theta_mean_f = F.leaky_relu(self.theta_mean_fc(h4), negative_slope = self.neg_slope)
            theta_var_f = F.leaky_relu(self.theta_var_fc(h4), negative_slope = self.neg_slope)

            backcast = self.backcast_out(theta_b)
            mean_forecast = self.mean_forecast_out(theta_mean_f)
            #var_forecast = F.softplus(self.var_forecast_out(theta_var_f), beta = 1)
            var_forecast = F.softplus(self.var_forecast_out(theta_var_f), beta = 1)**2
            #var_forecast = F.relu(self.var_forecast_out(theta_var_f)) + 1e-6

        else:

            h1 = F.leaky_relu(self.fc1(x.to(self.device)), negative_slope = self.neg_slope)
            h2 = F.leaky_relu(self.fc2(h1), negative_slope = self.neg_slope)
            h3 = F.leaky_relu(self.fc3(h2), negative_slope = self.neg_slope)
            h4 = F.leaky_relu(self.fc4(h3), negative_slope = self.neg_slope)

            theta_b = F.leaky_relu(self.theta_bc(h4), negative_slope = self.neg_slope)
            theta_mean_f = F.leaky_relu(self.theta_mean_fc(h4), negative_slope = self.neg_slope)
            theta_var_f = F.leaky_relu(self.theta_var_fc(h4), negative_slope = self.neg_slope)

            backcast = self.backcast_out(theta_b)
            mean_forecast = self.mean_forecast_out(theta_mean_f)
            #var_forecast = F.softplus(self.var_forecast_out(theta_var_f), beta = 1)
            var_forecast = F.softplus(self.var_forecast_out(theta_var_f), beta = 1)**2
            #var_forecast = F.relu(self.var_forecast_out(theta_var_f)) + 1e-6

        return backcast, mean_forecast, var_forecast


    def __str__(self):

        block_type = type(self).__name__

        return f'{block_type}(units={self.hidden_layer_units}, thetas_dims={self.thetas_dims}, ' \
            f'backcast_length={self.backcast_length}, ' \
            f'forecast_length={self.forecast_length}, share_thetas={self.share_thetas}, ' \
            f'dropout={self.dropout}, dropout_p={self.dropout_p}, neg_slope={self.neg_slope}) at @{id(self)}'

StableNBeatsGaussianNet:

In [ ]:
# Only the forward method is changed compared to standard NBeatsNet
class StableNBeatsGaussianNet(nn.Module):

    def __init__(self,
                 device,
                 backcast_length_multiplier,
                 forecast_length,
                 hidden_layer_units,
                 thetas_dims,
                 share_thetas,
                 nb_blocks_per_stack,
                 n_stacks,
                 share_weights_in_stack,
                 dropout = False,
                 dropout_p = 0.0,
                 neg_slope = 0.00):

        super().__init__()
        self.device = device
        self.backcast_length = backcast_length_multiplier * forecast_length
        self.forecast_length = forecast_length
        self.hidden_layer_units = hidden_layer_units
        self.thetas_dims = thetas_dims
        self.share_thetas = share_thetas
        self.nb_blocks_per_stack = nb_blocks_per_stack
        self.n_stacks = n_stacks
        self.share_weights_in_stack = share_weights_in_stack
        self.dropout = dropout
        self.dropout_p = dropout_p
        self.neg_slope = neg_slope

        self.stacks = []
        self.parameters = []

        print(f'| N-Beats')
        for stack_id in range(self.n_stacks):
            self.stacks.append(self.create_stack(stack_id))
        self.parameters = nn.ParameterList(self.parameters)
        self.to(self.device)


    def create_stack(self, stack_id):

        print(f'| --  Stack Generic (#{stack_id}) (share_weights_in_stack={self.share_weights_in_stack})')
        blocks = []
        for block_id in range(self.nb_blocks_per_stack):
            if self.share_weights_in_stack and block_id != 0:
                block = blocks[-1]  # pick up the last one when we share weights
            else:
                block = GenericNBeatsGaussianBlock(self.device,
                                                   self.backcast_length,
                                                   self.forecast_length,
                                                   self.hidden_layer_units,
                                                   self.thetas_dims,
                                                   self.share_thetas,
                                                   self.dropout,
                                                   self.dropout_p,
                                                   self.neg_slope)
                self.parameters.extend(block.parameters())
                print(f'     | -- {block}')
                blocks.append(block)

        return blocks


    def forward(self, backcast_arr):

        # dim backcast_arr = batch_size x shifts x backcast_length
        # shifts == 0 is standard input window, others are shifted lookback windows
        # higher index = further back in time
        # feed different input windows (per batch) through the SAME network (check via list of learnable parameters)
        # see https://stackoverflow.com/questions/54444630/application-of-nn-linear-layer-in-pytorch-on-additional-dimentions

        mean_forecast_arr = torch.zeros((backcast_arr.shape[0], # take batch size from backcast
                                         backcast_arr.shape[1], # take n of shifts from backcast
                                         self.forecast_length), dtype = torch.float).to(self.device)

        var_forecast_arr = torch.zeros((backcast_arr.shape[0], # take batch size from backcast
                                        backcast_arr.shape[1], # take n of shifts from backcast
                                        self.forecast_length), dtype = torch.float).to(self.device)

        # loop through stacks (and blocks)
        for stack_id in range(len(self.stacks)):
            for block_id in range(len(self.stacks[stack_id])):
                b, m_f, var_f = self.stacks[stack_id][block_id](backcast_arr)
                backcast_arr = backcast_arr.to(self.device) - b
                mean_forecast_arr = mean_forecast_arr.to(self.device) + m_f
                var_forecast_arr = var_forecast_arr.to(self.device) + var_f

        return backcast_arr, mean_forecast_arr, var_forecast_arr


# Training and evaluation

In [ ]:
def seed_torch(seed = 5101992):

    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

In [ ]:
class StableNBeatsGaussianLearner:

    def __init__(self,
                 device,
                 forecast_length,
                 configNBeats):

        gc.collect()

        self.device = device
        self.forecast_length = forecast_length
        self.configNBeats = configNBeats

        self.rndseed = self.configNBeats["rndseed"]

        seed_torch(self.rndseed)

        print('--- Model ---')
        self.model = StableNBeatsGaussianNet(self.device,
                                             self.configNBeats["backcast_length_multiplier"],
                                             self.forecast_length,
                                             self.configNBeats["hidden_layer_units"],
                                             self.configNBeats["thetas_dims"],
                                             self.configNBeats["share_thetas"],
                                             self.configNBeats["nb_blocks_per_stack"],
                                             self.configNBeats["n_stacks"],
                                             self.configNBeats["share_weights_in_stack"],
                                             self.configNBeats["dropout"],
                                             self.configNBeats["dropout_p"],
                                             self.configNBeats["neg_slope"])

        self.optim = torch.optim.Adam(self.model.parameters(),
                                      lr = self.configNBeats["learning_rate"],
                                      weight_decay = self.configNBeats["weight_decay"])

        self.init_state = copy.deepcopy(self.model.state_dict())
        self.init_state_opt = copy.deepcopy(self.optim.state_dict())

        self.model = self.model.to(self.device)
        wandb.watch(self.model)


    def ts_padding(self, ts_train_data, ts_eval_data):

        # Some time series in the dataset are not long enough to support the specified:
        # forecast_length and backcast_length + backcast input shifts
        # we use zero padding for the time series that are too short (neutral effect on loss calculations)
        # + self.shifts comes from the number of extra observations needed to create the shifted inputs/targets
        # + (self.forgins - 1) comes from rolling origin evaluation

        length_train = (self.configNBeats["backcast_length_multiplier"] * self.forecast_length +
                        self.forecast_length +
                        self.shifts)
        length_eval = (self.configNBeats["backcast_length_multiplier"] * self.forecast_length +
                       self.forecast_length +
                       self.shifts +
                       (self.forigins - 1))

        ts_train_pad = [x if x.size >= length_train else np.pad(x,
                                                                (int(length_train - x.size), 0),
                                                                'constant',
                                                                constant_values = 0) for x in ts_train_data]
        ts_eval_pad = [x if x.size >= length_eval else np.pad(x,
                                                              (int(length_eval - x.size), 0),
                                                              'constant',
                                                              constant_values = 0) for x in ts_eval_data]

        return ts_train_pad, ts_eval_pad


    def make_batch(self, batch_data, shuffle_origin = True):

        # If shuffle_origin = True --> batch for training --> random forecast origin based on LH
        # If shuffle_origin = False --> batch for evaluation --> fixed forecast origin

        # Split the batch into input_list and target_list
        # In x_arr and target_arr: batch x shift x backcats_length/forecast_length
        x_arr = np.empty(shape = (len(batch_data),
                                  self.shifts + 1,
                                  self.configNBeats["backcast_length_multiplier"] * self.forecast_length))
        target_arr = np.empty(shape = (len(batch_data),
                                       self.shifts + 1,
                                       self.forecast_length))

        # For every time series in the batch:
        # (1) slice the time series according to specific forecasting origin (depending on shuffle_origin)
        # (2) make shifted inputs/targets --> max number of shifts = forecast_length - 1
        # (3) fill x_arr and target_arr
        for j in range(len(batch_data)):
            i = batch_data[j]

            if shuffle_origin:
                # suffle_origin --> only in training


                # pick origin
                LH_max_offset = int(self.configNBeats["LH"] * self.forecast_length)
                ts_max_offset = int(len(i) -
                                    (self.configNBeats["backcast_length_multiplier"] * self.forecast_length +
                                     self.forecast_length +
                                     self.shifts))
                max_offset = min(LH_max_offset, ts_max_offset)
                if max_offset < 1:
                    offset = np.zeros(1)
                else:
                    offset = np.random.randint(low = 0, high = max_offset)
            else:
                offset = np.zeros(1)

            if offset == 0:
                for shift in range(self.shifts + 1):
                    if shift == 0:
                        x_arr[j, shift, :] = i[-self.forecast_length-self.configNBeats["backcast_length_multiplier"]*self.forecast_length:-self.forecast_length]
                        target_arr[j, shift, :] = i[-self.forecast_length:]
                    else:
                        x_arr[j, shift, :] = i[-self.forecast_length-self.configNBeats["backcast_length_multiplier"]*self.forecast_length-shift:-self.forecast_length-shift]
                        target_arr[j, shift, :] = i[-self.forecast_length-shift:-shift]
            else:
                for shift in range(self.shifts + 1):
                    x_arr[j, shift, :] = i[-self.forecast_length-self.configNBeats["backcast_length_multiplier"]*self.forecast_length-offset-shift:-self.forecast_length-offset-shift]
                    target_arr[j, shift, :] = i[-self.forecast_length-offset-shift:-offset-shift]

        scalers_arr = np.mean(x_arr, axis = (1,2))+1
        # Use same scaling factor for all shifted versions of time series
        # I think this is important for stability calculations!

        scalers_arr_reshaped = np.expand_dims(scalers_arr, (1,2))

        x_arr = x_arr / scalers_arr_reshaped
        target_arr = target_arr / scalers_arr_reshaped

        #print(target_arr)

        return x_arr, target_arr, scalers_arr_reshaped


    def create_example_plots(self, mean_forecast, var_forecast, target, actuals_train, final_evaluation = False):

        plot_forecasts = torch.cat((actuals_train, mean_forecast))
        plot_LQ1SD = torch.cat((actuals_train, mean_forecast - 1*torch.sqrt(var_forecast)))
        plot_LQ2SD = torch.cat((actuals_train, mean_forecast - 2*torch.sqrt(var_forecast)))
        plot_UQ1SD = torch.cat((actuals_train, mean_forecast + 1*torch.sqrt(var_forecast)))
        plot_UQ2SD = torch.cat((actuals_train, mean_forecast + 2*torch.sqrt(var_forecast)))
        plot_actuals = torch.cat((actuals_train, target))

        random_sample_forecasts = plot_forecasts.squeeze().numpy().tolist()
        random_sample_LQ1SD = plot_LQ1SD.squeeze().numpy().tolist()
        random_sample_LQ2SD = plot_LQ2SD.squeeze().numpy().tolist()
        random_sample_UQ1SD = plot_UQ1SD.squeeze().numpy().tolist()
        random_sample_UQ2SD = plot_UQ2SD.squeeze().numpy().tolist()
        random_sample_actuals = plot_actuals.squeeze().numpy().tolist()

        x_axis = torch.arange(1, len(random_sample_forecasts)+1).numpy().tolist()

        fig = go.Figure()

        #'''
        fig.add_traces(go.Scatter(x = x_axis,
                                  y = random_sample_UQ2SD,
                                  fill = None,
                                  mode = 'lines',
                                  name = '+2SD',
                                  line_color = 'rgba(255,0,0,0.2)',
                                  showlegend = False))

        fig.add_traces(go.Scatter(x = x_axis,
                                  y = random_sample_LQ2SD,
                                  fill = 'tonexty',
                                  mode = 'lines',
                                  name = '-2SD',
                                  line_color = 'rgba(255,0,0,0.2)',
                                  fillcolor = 'rgba(255,0,0,0.2)',
                                  showlegend = False))
        #'''

        fig.add_traces(go.Scatter(x = x_axis,
                                  y = random_sample_UQ1SD,
                                  fill = None,
                                  mode = 'lines',
                                  name = '+1SD',
                                  line_color = 'rgba(255,0,0,0.5)',
                                  showlegend = False))

        fig.add_traces(go.Scatter(x = x_axis,
                                  y = random_sample_LQ1SD,
                                  fill = 'tonexty',
                                  mode = 'lines',
                                  name = '-1SD',
                                  line_color = 'rgba(255,0,0,0.5)',
                                  fillcolor = 'rgba(255,0,0,0.5)',
                                  showlegend = False))

        fig.add_trace(go.Scatter(x = x_axis,
                                 y = random_sample_forecasts,
                                 mode = 'lines',
                                 name = 'forecasts',
                                 line_color = 'rgb(255,0,0)'))

        fig.add_trace(go.Scatter(x = x_axis,
                                 y = random_sample_actuals,
                                 mode = 'lines',
                                 name = 'actuals',
                                 line_color = 'rgb(0,0,0)'))

        # We only visualize examples for last epoch
        if not final_evaluation:
            wandb.log({"example_plots_evaluation": fig})
        else:
            wandb.log({"example_plots_final_evaluation": fig})


    def evaluate(self, x_arr, target_arr,
                 epoch = None,
                 need_grad = True,
                 early_stop = False):

        losses = dict()

        # Inputs must be converted to np.array of Tensors (float)
        x_arr = torch.from_numpy(x_arr).float().to(self.device)
        target_arr = torch.from_numpy(target_arr).float().to(self.device)

        if need_grad:
            self.model.train()
            self.model.to(self.device)
            _, mean_forecast_arr, var_forecast_arr = self.model(x_arr)

            NLL_shifts = 0.0
            for shift in range(self.shifts + 1):
                NLL_shifts += GaussianNLL(mean_forecast_arr[:, shift, :],
                                          var_forecast_arr[:, shift, :],
                                          target_arr[:, shift, :])
            losses["accuracy"] = NLL_shifts / (self.shifts + 1)

            if self.shifts > 0:
                # dimensions = batch_size x shifted forecast distributions for stability computations
                mean_forecast_base_arr = torch.zeros((mean_forecast_arr.shape[0],
                                                      sum(range(self.forecast_length - self.shifts, self.forecast_length))),
                                                     dtype = torch.float)
                var_forecast_base_arr = torch.zeros((var_forecast_arr.shape[0],
                                                     sum(range(self.forecast_length - self.shifts, self.forecast_length))),
                                                    dtype = torch.float)
                mean_forecast_shift_arr = torch.zeros((mean_forecast_arr.shape[0],
                                                       sum(range(self.forecast_length - self.shifts, self.forecast_length))),
                                                      dtype = torch.float)
                var_forecast_shift_arr = torch.zeros((var_forecast_arr.shape[0],
                                                      sum(range(self.forecast_length - self.shifts, self.forecast_length))),
                                                     dtype = torch.float)
                col = 0
                for shift in range(1, self.shifts + 1):
                    for horizon_m1 in range(self.forecast_length - shift):
                        mean_forecast_base_arr[:, col] = mean_forecast_arr[:, 0, horizon_m1]
                        var_forecast_base_arr[:, col] = var_forecast_arr[:, 0, horizon_m1]
                        mean_forecast_shift_arr[:, col] = mean_forecast_arr[:, shift, horizon_m1 + shift]
                        var_forecast_shift_arr[:, col] = var_forecast_arr[:, shift, horizon_m1 + shift]
                        col = col + 1
                
                
                if self.configNBeats["training_stability"] == "WD":
                    
                    #print("WD")
                    losses["stability"] = GaussianWD(mean_forecast_base_arr, var_forecast_base_arr,
                                                         mean_forecast_shift_arr, var_forecast_shift_arr)
                elif self.configNBeats["training_stability"] == "KL":
                    #print("KL")
                    losses["stability"] = (0.5 * GaussianKL(mean_forecast_base_arr, var_forecast_base_arr,
                                                            mean_forecast_shift_arr, var_forecast_shift_arr) +
                                           0.5 * GaussianKL(mean_forecast_shift_arr, var_forecast_shift_arr,
                                                            mean_forecast_base_arr, var_forecast_base_arr))
                elif self.configNBeats["training_stability"] == "RMSC":
                    #print("RMSC")
                    losses["stability"] = RMSE(mean_forecast_base_arr, mean_forecast_shift_arr)
            else:
                losses["stability"] = torch.zeros(1)

        else:
            with torch.no_grad():
                self.model.eval()
                self.model.to(self.device)
                _, mean_forecast_arr, var_forecast_arr = self.model(x_arr)

                NLL_shifts = 0.0
                for shift in range(self.shifts + 1):
                    NLL_shifts += GaussianNLL(mean_forecast_arr[:, shift, :],
                                              var_forecast_arr[:, shift, :],
                                              target_arr[:, shift, :])
                losses["accuracy"] = NLL_shifts / (self.shifts + 1)

                if self.shifts > 0:
                    mean_forecast_base_arr = torch.zeros((mean_forecast_arr.shape[0],
                                                      sum(range(self.forecast_length - self.shifts, self.forecast_length))),
                                                     dtype = torch.float)
                    var_forecast_base_arr = torch.zeros((var_forecast_arr.shape[0],
                                                         sum(range(self.forecast_length - self.shifts, self.forecast_length))),
                                                        dtype = torch.float)
                    mean_forecast_shift_arr = torch.zeros((mean_forecast_arr.shape[0],
                                                           sum(range(self.forecast_length - self.shifts, self.forecast_length))),
                                                          dtype = torch.float)
                    var_forecast_shift_arr = torch.zeros((var_forecast_arr.shape[0],
                                                          sum(range(self.forecast_length - self.shifts, self.forecast_length))),
                                                         dtype = torch.float)
                    col = 0
                    for shift in range(1, self.shifts + 1):
                        for horizon_m1 in range(self.forecast_length - shift):
                            mean_forecast_base_arr[:, col] = mean_forecast_arr[:, 0, horizon_m1]
                            var_forecast_base_arr[:, col] = var_forecast_arr[:, 0, horizon_m1]
                            mean_forecast_shift_arr[:, col] = mean_forecast_arr[:, shift, horizon_m1 + shift]
                            var_forecast_shift_arr[:, col] = var_forecast_arr[:, shift, horizon_m1 + shift]
                            col = col + 1
                    
                    if self.configNBeats["training_stability"] == "WD":
                        
                        losses["stability"] = GaussianWD(mean_forecast_base_arr, var_forecast_base_arr,
                                                         mean_forecast_shift_arr, var_forecast_shift_arr)
                    elif self.configNBeats["training_stability"] == "KL":
                        losses["stability"] = (0.5 * GaussianKL(mean_forecast_base_arr, var_forecast_base_arr,
                                                            mean_forecast_shift_arr, var_forecast_shift_arr) +
                                           0.5 * GaussianKL(mean_forecast_shift_arr, var_forecast_shift_arr,
                                                            mean_forecast_base_arr, var_forecast_base_arr))
                    elif self.configNBeats["training_stability"] == "RMSC":
                        losses["stability"] = RMSE(mean_forecast_base_arr, mean_forecast_shift_arr)
                else:
                    losses["stability"] = torch.zeros(1)

                if not self.disable_plot:
                    # Plot validation examples - of standard/unshifted input - for last epoch
                    # This part of the evaluation function is called after training has been completed
                    if (epoch == self.configNBeats["epochs"]):
                        self.create_example_plots(mean_forecast_arr[0, 0, :], var_forecast_arr[0, 0, :],
                                                  target_arr[0, 0, :], x_arr[0, 0, :])

        return losses


    # Training of net (training data can include validation data) + validation or testing
    def train_net(self,
                  ts_train_mm,
                  ts_eval_mm,
                  forigins,
                  validation = True,
                  disable_plot = True):

        self.forigins = forigins
        self.shifts = self.configNBeats["shifts"]
        self.validation = validation
        self.disable_plot = disable_plot
        #assert self.shifts < self.forecast_length # max allowed number of shifts is forecast_length - 1

        # Data preprocessing depends on backcast_length_multiplier
        ts_train_pad, ts_eval_pad = self.ts_padding(ts_train_mm, ts_eval_mm)
        ts_train_pad = np.array(ts_train_pad, dtype = object)
        ts_eval_pad = np.array(ts_eval_pad, dtype = object)

        print('--- Training ---')

        # Containers to save train/evaluation losses and parameters
        tloss_combined, tloss_accuracy, tloss_stability = [], [], []
        eloss_combined, eloss_accuracy, eloss_stability = [], [], []
        #params = []

        # Main training loop
        self.model.load_state_dict(self.init_state)
        self.optim.load_state_dict(self.init_state_opt)

        seed_torch(self.rndseed)

        for epoch in range(1, self.configNBeats["epochs"]+1):

            start_time = time()
            # Shuffle train data
            np.random.shuffle(ts_train_pad)
            # Determine number of batches per epoch
            num_batches = int(ts_train_pad.shape[0] / self.configNBeats["batch_size"])

            # Training per epoch
            avg_tloss_combined_epoch = 0.0
            avg_tloss_accuracy_epoch = 0.0
            avg_tloss_stability_epoch = 0.0

            for k in range(num_batches):

                batch = np.array(ts_train_pad[k*self.configNBeats["batch_size"]:(k+1)*self.configNBeats["batch_size"]])
                x_arr, target_arr, _ = self.make_batch(batch, shuffle_origin = True)

                self.optim.zero_grad()
                losses_batch = self.evaluate(x_arr, target_arr,
                                             epoch, need_grad = True,
                                             early_stop = False)
                if self.shifts > 0:
                    loss_combined = ((self.configNBeats["lambda"] * losses_batch["stability"]) +
                                     ((1 - self.configNBeats["lambda"]) * losses_batch["accuracy"]))
                else:
                    loss_combined = losses_batch["accuracy"]
                loss_combined.backward()

                self.optim.step()

                #params = self.model.parameters()
                #total_params = sum(p.numel() for p in self.model.parameters() if p.requires_grad)
                #if (epoch == 1 or epoch == self.configNBeats["epochs"]) and k == 0:
                #    print('Epoch {}/{} \t n_learnable_pars={:.4f}'.format(
                #        epoch,
                #        self.configNBeats["epochs"],
                #        total_params))

                wandb.log({"tloss_comb_step": loss_combined,
                           "tloss_acc_step": losses_batch["accuracy"],
                           "tloss_stab_step": losses_batch["stability"]})

                avg_tloss_combined_epoch += (loss_combined / num_batches)
                avg_tloss_accuracy_epoch += (losses_batch["accuracy"] / num_batches)
                avg_tloss_stability_epoch += (losses_batch["stability"] / num_batches)


            elapsed_time = time() - start_time

            print('Epoch {}/{} \t tloss_combined={:.4f} \t time={:.2f}s'.format(
                epoch,
                self.configNBeats["epochs"],
                avg_tloss_combined_epoch,
                elapsed_time))

            wandb.log({"epoch": epoch,
                        "tloss_comb_evol": avg_tloss_combined_epoch,
                        "tloss_acc_evol": avg_tloss_accuracy_epoch,
                        "tloss_stab_evol": avg_tloss_stability_epoch})

        wandb.log({"tloss_comb": avg_tloss_combined_epoch,
                   "tloss_acc": avg_tloss_accuracy_epoch,
                   "tloss_stab": avg_tloss_stability_epoch})

        print('--- Training done ---')
        print('--- Final evaluation ---')

        # Containers to save actuals and forecasts
        #'''
        actuals = np.empty(shape = (len(ts_eval_pad), self.forigins, self.forecast_length)) # n_series, forigin, forecast_length
        mean_forecasts = np.empty(shape = (len(ts_eval_pad), self.forigins, self.forecast_length)) # n_series, forigin, forecast_length
        mean_forecasts_shifted = np.empty(shape = (len(ts_eval_pad), self.forigins, self.forecast_length)) # n_series, forigin, forecast_length
        var_forecasts = np.empty(shape = (len(ts_eval_pad), self.forigins, self.forecast_length)) # n_series, forigin, forecast_length
        var_forecasts_shifted = np.empty(shape = (len(ts_eval_pad), self.forigins, self.forecast_length)) # n_series, forigin, forecast_length
        actuals_train = np.empty(shape = (len(ts_eval_pad), self.forigins,
                                          self.configNBeats["backcast_length_multiplier"] * self.forecast_length)) # n_series, forigin, backcast_length
        #'''

        actuals_scaled = np.empty(shape = (len(ts_eval_pad), self.forigins, self.forecast_length)) # n_series, forigin, forecast_length
        actuals_shifted_scaled = np.empty(shape = (len(ts_eval_pad), self.forigins, self.forecast_length)) # n_series, forigin, forecast_length
        mean_forecasts_scaled = np.empty(shape = (len(ts_eval_pad), self.forigins, self.forecast_length)) # n_series, forigin, forecast_length
        mean_forecasts_shifted_scaled = np.empty(shape = (len(ts_eval_pad), self.forigins, self.forecast_length)) # n_series, forigin, forecast_length
        var_forecasts_scaled = np.empty(shape = (len(ts_eval_pad), self.forigins, self.forecast_length)) # n_series, forigin, forecast_length
        var_forecasts_shifted_scaled = np.empty(shape = (len(ts_eval_pad), self.forigins, self.forecast_length)) # n_series, forigin, forecast_length
        actuals_train_scaled = np.empty(shape = (len(ts_eval_pad), self.forigins,
                                                 self.configNBeats["backcast_length_multiplier"] * self.forecast_length)) # n_series, forigin, backcast_length

        # Forecasts for each origin in rolling_window
        for forigin in range(self.forigins):

            # Only one batch, but one batch per forecast origin
            if forigin < self.forigins-1:
                eval_data_subset = np.array([x[:(-18 + forigin + self.forecast_length)] for x in ts_eval_pad],
                                           dtype = object)
            else:
                eval_data_subset = np.array([x for x in ts_eval_pad], dtype = object)

            x_arr, target_arr, scalers_arr_reshaped = self.make_batch(eval_data_subset, shuffle_origin = False)

            # Produce forecasts for subset of test data
            x_arr = torch.from_numpy(x_arr).float().to(self.device)
            target_arr = torch.from_numpy(target_arr).float().to(self.device)
            with torch.no_grad():
                self.model.eval()
                self.model.to(self.device)
                _, mean_forecast_arr, var_forecast_arr = self.model(x_arr)

            #RESCALING#######################################################################################
            scalers = torch.from_numpy(scalers_arr_reshaped).float().to(self.device)

            mean_forecast_arr_scaled = mean_forecast_arr * scalers
            var_forecast_arr_scaled = var_forecast_arr * scalers**2

            target_arr_scaled = target_arr * scalers
            x_arr_scaled = x_arr * scalers
            #RESCALING#######################################################################################

            # Plot 10 random examples per origin - of standard/unshifted input
            #sample_ids = np.random.randint(low = 0, high = int(x_arr_scaled.shape[0]), size = 10)
            sample_ids = np.arange(2)#10)
            for sample_id in sample_ids:
                '''
                self.create_example_plots(mean_forecast_arr[sample_id, 0, :],
                                          var_forecast_arr[sample_id, 0, :],
                                          target_arr[sample_id, 0, :],
                                          x_arr[sample_id, 0, :],
                                          final_evaluation = True)
                '''
                self.create_example_plots(mean_forecast_arr_scaled[sample_id, 0, :].cpu(),
                                          var_forecast_arr_scaled[sample_id, 0, :].cpu(),
                                          target_arr_scaled[sample_id, 0, :].cpu(),
                                          x_arr_scaled[sample_id, 0, :].cpu(),
                                          final_evaluation = True)

            # Save to containers
            #'''
            mean_forecasts[:, forigin, :] = mean_forecast_arr[:, 0, :].cpu()
            mean_forecasts_shifted[:, forigin, :] = mean_forecast_arr[:, 1, :].cpu()
            var_forecasts[:, forigin, :] = var_forecast_arr[:, 0, :].cpu()
            var_forecasts_shifted[:, forigin, :] = var_forecast_arr[:, 1, :].cpu()
            actuals[:, forigin, :] = target_arr[:, 0, :].cpu()
            actuals_train[:, forigin, :] = x_arr[:, 0, :].cpu()
            #'''

            mean_forecasts_scaled[:, forigin, :] = mean_forecast_arr_scaled[:, 0, :].cpu()
            mean_forecasts_shifted_scaled[:, forigin, :] = mean_forecast_arr_scaled[:, 1, :].cpu()
            var_forecasts_scaled[:, forigin, :] = var_forecast_arr_scaled[:, 0, :].cpu()
            var_forecasts_shifted_scaled[:, forigin, :] = var_forecast_arr_scaled[:, 1, :].cpu()
            actuals_scaled[:, forigin, :] = target_arr_scaled[:, 0, :].cpu()
            actuals_shifted_scaled[:, forigin, :] = target_arr_scaled[:, 1, :].cpu()
            actuals_train_scaled[:, forigin, :] = x_arr_scaled[:, 0, :].cpu()

    
        # Compute accuracy

        smape = SMAPE(mean_forecasts_scaled, actuals_scaled)
        smapc = SMAPE(mean_forecasts_scaled[: , :, :-1], mean_forecasts_shifted_scaled[: , :, 1:])
        mape = MAPE(mean_forecasts_scaled, actuals_scaled)
        mapc = MAPE(mean_forecasts_scaled[: , :, :-1], mean_forecasts_shifted_scaled[: , :, 1:])

        rmsse = RMSSE(mean_forecasts_scaled, actuals_scaled, actuals_train_scaled)
        rmssc = RMSSE(mean_forecasts_scaled[: , :, :-1], mean_forecasts_shifted_scaled[: , :, 1:], actuals_train_scaled)

        crps = sCRPS(mean_forecasts_scaled, var_forecasts_scaled, actuals_scaled, actuals_train_scaled)

        kl = (0.5 * GaussianKL_eval(mean_forecasts_scaled[: , :, :-1], var_forecasts_scaled[: , :, :-1],
                                    mean_forecasts_shifted_scaled[: , :, 1:], var_forecasts_shifted_scaled[: , :, 1:]) +
              0.5 * GaussianKL_eval(mean_forecasts_shifted_scaled[: , :, 1:], var_forecasts_shifted_scaled[: , :, 1:],
                                    mean_forecasts_scaled[: , :, :-1], var_forecasts_scaled[: , :, :-1]))

        print('sMAPE={:.4f} \t sMAPC={:.4f}'.format(smape, smapc))
        print('MAPE={:.4f} \t MAPC={:.4f}'.format(mape, mapc))
        print('RMSSE={:.4f} \t RMSSC={:.4f}'.format(rmsse, rmssc))
        print('CRPS={:.4f}'.format(crps))
        print('KL={:.4f}'.format(kl))

        wandb.log({"sMAPE": smape, "sMAPC": smapc,
                   "MAPE": mape, "MAPC": mapc,
                   "RMSSE": rmsse, "RMSSC": rmssc,
                   "CRPS": crps,
                   "KL": kl})

        # n_series, forigin, forecast_length
        fc_colnames = [str(i) for i in range(1, self.forecast_length + 1)]

        actuals_np = actuals#.numpy()
        actuals_np = actuals_scaled#.numpy()
        m,n,r = actuals_np.shape
        actuals_arr = np.column_stack((np.repeat(np.arange(m) + 1, n),
                                       np.tile(np.arange(n) + 1, m),
                                       actuals_np.reshape(m*n, -1)))
        actuals_df = pd.DataFrame(actuals_arr, columns = ['item_id', 'fc_origin'] + fc_colnames)
        helper_col = ['actual'] * len(actuals_df)
        actuals_df['type'] = helper_col

        mean_forecasts_np = mean_forecasts#.numpy()
        mean_forecasts_np = mean_forecasts_scaled#.numpy()
        m,n,r = mean_forecasts_np.shape
        mean_forecasts_arr = np.column_stack((np.repeat(np.arange(m) + 1, n),
                                              np.tile(np.arange(n) + 1, m),
                                              mean_forecasts_np.reshape(m*n, -1)))
        mean_forecasts_df = pd.DataFrame(mean_forecasts_arr, columns = ['item_id', 'fc_origin'] + fc_colnames)
        helper_col = ['mean_forecast'] * len(mean_forecasts_df)
        mean_forecasts_df['type'] = helper_col

        sd_forecasts_np = np.sqrt(var_forecasts)#.numpy()
        sd_forecasts_np = np.sqrt(var_forecasts_scaled)#.numpy()
        m,n,r = sd_forecasts_np.shape
        sd_forecasts_arr = np.column_stack((np.repeat(np.arange(m) + 1, n),
                                            np.tile(np.arange(n) + 1, m),
                                            sd_forecasts_np.reshape(m*n, -1)))
        sd_forecasts_df = pd.DataFrame(sd_forecasts_arr, columns = ['item_id', 'fc_origin'] + fc_colnames)
        helper_col = ['sd_forecast'] * len(sd_forecasts_df)
        sd_forecasts_df['type'] = helper_col

        output_df_mm = pd.concat([actuals_df, mean_forecasts_df, sd_forecasts_df])

        wandb.join()

        return output_df_mm

# Run experiments

Setup:

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(device)

wandb_project_name = 'nbeats_stability_prob'
job_type_name = 'test_M4M' # one of 'validation_M3/4M' or 'test_M3/4M'

hyperparameter_defaults = dict()
hyperparameter_defaults['epochs'] = 155
hyperparameter_defaults['batch_size'] = 512
hyperparameter_defaults['nb_blocks_per_stack'] = 1
hyperparameter_defaults['thetas_dims'] = 256
hyperparameter_defaults['n_stacks'] = 20
hyperparameter_defaults['share_weights_in_stack'] = False
hyperparameter_defaults["backcast_length_multiplier"] = 4
hyperparameter_defaults['hidden_layer_units'] = 256
hyperparameter_defaults['share_thetas'] = False
hyperparameter_defaults["dropout"] = False
hyperparameter_defaults["dropout_p"] = 0.0
hyperparameter_defaults["neg_slope"] = 0.00
hyperparameter_defaults['learning_rate'] = 0.0005
hyperparameter_defaults["weight_decay"] = 0.00
hyperparameter_defaults["LH"] = 10
hyperparameter_defaults["rndseed"] = 0 #seeds used for paper: [1500,2500,3500,4500,5500,6500,7500,8500,9500,10500]
hyperparameter_defaults["shifts"] = 1
hyperparameter_defaults['patience'] = 300 # Only affects validation runs
hyperparameter_defaults['lambda'] = 0.15 # Rel. weight of forecast stability loss in loss_combined
                                         # --> lambda_code = lambda_paper/(1+lambda_paper)
                                         #e.g., if lambda_paper= 0.176 --> lambda_coda=0.15
hyperparameter_defaults['training_stability'] = 'KL' #'WD' 'KL' 'RMSC'

In [ ]:
if job_type_name == 'test_M3M':
    is_val = False
    mm_train, mm_eval = valset_m3m, testset_m3m
elif job_type_name == 'validation_M3M':
    is_val = True
    mm_train, mm_eval = trainset_m3m, valset_m3m
elif job_type_name == 'test_M4M':
    is_val = False
    mm_train, mm_eval = valset_m4m, testset_m4m
elif job_type_name == 'validation_M4M':
    is_val = True
    mm_train, mm_eval = trainset_m4m, valset_m4m

Run:

In [ ]:
def sweep_function():
    wandb.init(config = hyperparameter_defaults,
              project = wandb_project_name,
              job_type = job_type_name)
    config = wandb.config
    run_name = wandb.run.name
    
    # Initialize model
    StableNBeatsGaussian_model = StableNBeatsGaussianLearner(device, 6, config)
    # Train & evaluate
    df_save = StableNBeatsGaussian_model.train_net(mm_train, mm_eval, 13, is_val)

    df_save.to_csv('nbeats_stability_prob' + job_type_name + '_' + run_name + '.csv', index = False)

In [ ]:
sweep_config = {
  "name": "m4_lambda",
  "method": "grid",
  "parameters": {
        "lambda": {
            "values": [0]
        },
      "rndseed": {
            "values": [1500,2500,3500,4500,5500,6500,7500,8500,9500,10500]
      },
      "training_stability": {
            "values": ["RMSC"] 
        }
    }
}

sweep_id = wandb.sweep(sweep_config, project = wandb_project_name)

In [ ]:
wandb.agent(sweep_id, function = sweep_function)